# 3.5 Colocando o Istio em execução

## Instalando o Istio (linux)

Para instalar a última versão do Istio, neste momento 1.7.3, você pode ir até a página [Getting Started](https://istio.io/latest/docs/setup/getting-started/#download) ou seguir as instruções abaixo:

In [ ]:
curl -L https://istio.io/downloadIstio | sh -

Para usar o comando `istioctl`, que está no diretório `bin` do download, coloque-o na variável `PATH` ou copie o arquivo `bin/istioctl` para um diretório no seu `PATH`

In [ ]:
# Verificar onde está o executável do Istio
ls istio-1.7.3/bin
# Verificar o conteúdo da variável PATH
echo $PATH
# Vamos coloca-lo no diretório bin do usuário
[ -d "$HOME/bin" ] && echo "O diretorio $HOME/bin existe."
# Copiar o arquivo para o diretório bin
cp istio-1.7.3/bin/istioctl ~/bin
# Teste
istioctl --help

Você pode copiar o comando em qualquer diretório ou mantê-lo e ajustar a variável `PATH` na sessão.

In [ ]:
export PATH=$PWD/istio-1.7.3/bin:$PATH
echo $PATH
istioctl --help